In [1]:
%run Main.ipynb

# Redshift Drift 

## - Spectroscopic velocity

$\Delta v = \frac{c \Delta z}{1+z} = c H_0 \Delta t \left[1 - \frac{E(z)}{1+z}\right] = kh \left[1 - \frac{E(z)}{1+z}\right]$, where $H_0 = 100 H_{100}$ and $k = c H_{100}\delta t$

Assuming $\delta t = 1$ year we have $k=3.064$ cm/s.

In [2]:
def spect_velocity(zs, h, OmegaM, w0, wa, NrYears):
    if w0 == None: 
        w0 = -1.
    if wa == None: 
        wa = 0
    return c100t1 * NrYears * h * ( 1 - 1/(1+zs) * function_E_z(zs, OmegaM, w0, wa))

## - Spectroscopic velocity uncertainty

#### Liske Approach - (Approximation of the spectroscopic velocity uncertainty for ELT)
$\sigma_{\Delta v} = 1.35 \frac{2370}{S/N} \sqrt{\frac{30}{N_{QSO}})} \left({\frac{5}{1+z_{QSO}}}\right)^x$, with $x=1.7$ for $z\leqslant4$ and $x=0.9$ for $z>4$ 

In [3]:
def uncLiske_spect_velocity(zs):
    NrQSO = 6
    SN    = 3000
    unc_zELT = np.zeros_like(zs)
    for i in np.arange(len(zs)):
        if zs[i] <= 4:
            x = 1.7
        else:
            x = .9
        unc_zELT[i] = 1.35 * 2370/SN * np.sqrt(30/NrQSO) * (5/(1+zs[i]))**x
    return unc_zELT

## - Derivatives calculation

In [4]:
def dfdh_z(z, h, OmegaM, w0, wa, NrYears):
    return ( spect_velocity(z, h+hh, OmegaM, w0, wa, NrYears)-spect_velocity(z, h, OmegaM, w0, wa, NrYears) )/hh

def dfdOmegaM_z(z, h, OmegaM, w0, wa, NrYears):
    return ( spect_velocity(z, h,OmegaM+hh, w0, wa, NrYears)-spect_velocity(z, h, OmegaM, w0, wa, NrYears) )/hh

def dfdw0_z(z, h, OmegaM, w0, wa, NrYears):
    if w0 != None:
        return ( spect_velocity(z, h, OmegaM, w0+hh, wa, NrYears)-spect_velocity(z, h, OmegaM, w0, wa, NrYears) )/hh
    else:
        return 0

def dfdwa_z(z, h, OmegaM, w0, wa, NrYears):
    if wa != None:
        return ( spect_velocity(z, h,OmegaM, w0, wa+hh, NrYears)-spect_velocity(z, h, OmegaM, w0, wa, NrYears) )/hh
    else:
        return 0

# Measurements
### - Redshifts

In [5]:
zLiske = np.array([2., 2.5, 3., 3.5, 4.5])

### - Uncertainties

In [6]:
unc_zLiske = uncLiske_spect_velocity(zLiske)

# Priors

In [7]:
unch = 0.05
uncOmegaM = 0.03
uncw0 = 0.1
uncwa = 0.4

# Fiducial Models

<b>- Freezing CPL:</b> CPLF $(w_0 = −0.9, w_a = +0.3)$

<b>- Thawing CPL:</b> CPLT $(w_0 = −0.8, w_a = −0.2)$

# Fisher Matrices
## - No Priors

In [8]:
CPLT_Liske_np = FisherMatrix_z(zLiske, unc_zLiske, h, OmegaM, w0=-0.8, wa=-0.2, NrYears=20, analysis = 'redshift')
CPLF_Liske_np = FisherMatrix_z(zLiske, unc_zLiske, h, OmegaM, w0=-0.9, wa=0.3, NrYears=20, analysis = 'redshift')

## - With Priors

In [9]:
#Separate priors in h,OmegaM,w0,wa 
CPLT_Liske = add1DPriors(CPLT_Liske_np, [[unch,0],[uncOmegaM,1],[uncw0,2],[uncwa,3]])
CPLF_Liske = add1DPriors(CPLF_Liske_np, [[unch,0],[uncOmegaM,1],[uncw0,2], [uncwa,3]])

# Confidence ellipses (1-sigma)

In [10]:
def EllipsesRed(parameter1,parameter2, parameter_values, Fs, ells_color, ells_linestyle, Fs_legend=None): 
    ells = []
    custom_lines=[]
    if not Fs_legend:
        Fs_legend = len(Fs)*[None]
    for i in range(len(Fs)):
        parameter1_value, parameter2_value = parameter_values[i] 
        
        if parameter1 == 'w0':
            parameter1_legend = '$w_0$'
            
            if parameter2 == 'wa':
                Fm = marginalize(Fs[i],'OmegaM',analysis='redshift')
                Fm = marginalize(Fm,'h',analysis='redshift')
                parameter2_legend = '$w_a$' 
                
            if parameter2 == 'h':
                Fm = marginalize(Fs[i],'wa',analysis='redshift')
                Fm = marginalize(Fm,'OmegaM',analysis='redshift')
                parameter2_legend = '$h$'
    
        
        if parameter1 == 'OmegaM':
            parameter1_legend = '$\Omega_m$'
        
            if parameter2 == 'w0':
                Fm = marginalize(Fs[i],'wa',analysis='redshift')
                Fm = marginalize(Fm,'h',analysis='redshift')
                parameter2_legend = '$w_0$'
                
            if parameter2 == 'wa':
                Fm = marginalize(Fs[i],'w0',analysis='redshift')
                Fm = marginalize(Fm,'h',analysis='redshift')
                parameter2_legend = '$w_a$'
            
            
        if parameter1 == 'wa' and parameter2 == 'h':
            parameter1_legend = '$w_a$'
            parameter2_legend = '$h$'
            Fm = marginalize(Fs[i],'w0',analysis='redshift')
            Fm = marginalize(Fm,'OmegaM',analysis='redshift')
              
            
        if parameter1 == 'h' and parameter2 == 'OmegaM':
            parameter1_legend = '$h$'
            parameter2_legend = '$\Omega_m$'
            Fm = marginalize(Fs[i],'wa',analysis='redshift')
            Fm = marginalize(Fm,'w0',analysis='redshift')
            
        
        Pa = preelipse(Fm)
        color, name, style,facecolor, alpha = ells_color[i], Fs_legend[i], ells_linestyle[i], 'none', 1
        ells.append([parameter1_value,parameter2_value,Pa[0],Pa[1],Pa[2],name,color,style,facecolor, alpha])
        custom_lines+= [Line2D([0], [0], color=ells_color[i], lw=1.1, ls=ells_linestyle[i])]
        
                            
    ellipses(ells,[parameter1_legend,parameter2_legend])
    plt.tick_params(axis='both', which='major', labelsize=16)
    if Fs_legend[0]: 
        plt.legend(custom_lines, Fs_legend)
    
    #Definition of plot limits and figure name
    #plt.xlim(-1.1,-0.7)
    #plt.ylim(-0.4, 1.0)
    #plt.savefig('name.png', dpi=300, bbox_inches='tight')

    plt.show()

The possible combinations are: (h,OmegaM), (OmegaM, w0), (OmegaM, wa), (w0, wa), (w0, h), (wa, h)

Specify the <b>combination of parameters</b>, <b>fiducial values</b> of the model, the <b>matrices</b>, <b>colours</b>, <b>linestyle</b> and <b>names</b> for the legend.

In [11]:
#EllipsesRed('h','OmegaM',[[0.7,0.3],[0.7,0.3]],[CPLT_Liske, CPLF_Liske], ['blue','red'], ['-', (0, (8,8))],['Thawing', 'Freezing'])

# Table with ρ, FoM and σ

In [12]:
def TableRed(matrices):
    i=0
    df=pd.DataFrame()
    nformat = "{:.0f}"
    float_format="{:.3f}"
    
    
    for matrix in matrices:
        
        rows = {'ρ(h, Ωm)':float_format.format(FigMeriteRho(matrix,['w0','wa'],analysis='redshift')[1]),
                'ρ(Ωm, w0)':float_format.format(FigMeriteRho(matrix,['h','wa'],analysis='redshift')[1]),
                'ρ(h, w0)':float_format.format(FigMeriteRho(matrix,['OmegaM','wa'],analysis='redshift')[1]),                       
                'ρ(Ωm, wa)':float_format.format(FigMeriteRho(matrix,['h','w0'],analysis='redshift')[1]),
                'ρ(h, wa)':float_format.format(FigMeriteRho(matrix,['OmegaM','w0'],analysis='redshift')[1]),
                'ρ(w0, wa)':float_format.format(FigMeriteRho(matrix,['h','OmegaM'],analysis='redshift')[1]),
                'FoM(h, Ωm)': nformat.format(FigMeriteRho(matrix,['w0','wa'],analysis='redshift')[0]),
                'FoM(Ωm, w0)': nformat.format(FigMeriteRho(matrix,['h','wa'],analysis='redshift')[0]),
                'FoM(h, w0)': nformat.format(FigMeriteRho(matrix,['OmegaM','wa'],analysis='redshift')[0]),
                'FoM(Ωm, wa)': nformat.format(FigMeriteRho(matrix,['h','w0'],analysis='redshift')[0]),
                'FoM(h, wa)': nformat.format(FigMeriteRho(matrix,['OmegaM','w0'],analysis='redshift')[0]),
                'FoM(w0, wa)': nformat.format(FigMeriteRho(matrix,['h','OmegaM'],analysis='redshift')[0]),
                'σ(h)':float_format.format(Sigmas(matrix)[0]),
                'σ(Ωm)':float_format.format(Sigmas(matrix)[1]),
                'σ(w0)':float_format.format(Sigmas(matrix)[2]),
                'σ(wa)':float_format.format(Sigmas(matrix)[3])}
        
        i+=1
        df = df.append(rows,ignore_index=True)
        del rows
            
    Datasets = ['Thawing CPL', 'Freezing CPL']
    df.insert(0, "Parameters", Datasets, True)
    
    df = df.transpose()
    
            
    return df

In [13]:
matricesCPL = [CPLT_Liske, CPLF_Liske]
#tab = TableRed(matricesCPL)

#See the table in DataFrame:
#tab

#Transform dataframe to latex code:
#print(tab.to_latex(header=False,index=True))